# solution_dataset_1_and_2

In [190]:
# -*- coding: utf-8 -*-

# -- Sheet 2 --

# **############# Imports, settings and first dataset view ###############**


# Imports, settings and first dataset view
import pandas as pd
import seaborn as sns
import numpy as np
import json

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter

# Set pandas to show all columns when you print a dataframe
pd.set_option('display.max_columns', None)

# Global setting here you choose the dataset number and classification type for the model
dataset_number = 2  # Options are [1, 2, 3, 4]
test_type = 'label'  # Options are ['label', 'attack_type']

# Read the json and read it to a pandas dataframe object, you can change these settings
with open(f'combined_datasets_for_students/dataset_{str(dataset_number)}_train.json') as file:
    raw_ds = json.load(file)
df = pd.json_normalize(raw_ds, max_level=2)

# Shoe the first five lines of the dataframe to see if everything was read accordingly 
df

# **# open the dataset file**


# Fill the black attack tag lines with "Benign" string
df['request.Attack_Tag'] = df['request.Attack_Tag'].fillna('Benign')
df['attack_type'] = df['request.Attack_Tag']


# This function will be used in the lambda below to iterate over the label columns 
# You can use this snippet to run your own lambda on any data with the apply() method
def categorize(row):
    if row['request.Attack_Tag'] == 'Benign':
        return 'Benign'
    return 'Malware'


df['label'] = df.apply(lambda row: categorize(row), axis=1)

# After finishing the arrangements we delete the irrelevant column
df.drop('request.Attack_Tag', axis=1, inplace=True)

df

# **############## Basic dataset label arrangements ###############**


# Remove all NAN columns or replace with desired string
# This loop iterates over all of the column names which are all NaN
for column in df.columns[df.isna().any()].tolist():
    # df.drop(column, axis=1, inplace=True)
    df[column] = df[column].fillna('None')

# If you want to detect columns that may have only some NaN values use this:
# df.loc[:, df.isna().any()].tolist()


# Setting features for further feature extraction by choosing columns
# Some will be "simply" encoded via label encoding and others with HashingVectorizer

# On these headers we will run a "simple" BOW
SIMPLE_HEADERS = [
    #   'request.headers.Host',
    #   'request.headers.Accept',
    #   'request.headers.Connection',
    #   'request.headers.Sec-Fetch-User',
    # 'response.headers.Content-Type',
    #   'request.body',
    #   'response.headers.Content-Length',
    # 'request.headers.Accept-Encoding',
    #   'request.method',
    # 'request.headers.Accept-Language',
    # 'request.headers.Sec-Fetch-Site',
    #   'request.headers.Sec-Fetch-Mode',
    # 'request.headers.Sec-Fetch-Dest',
    #   'response.status',
    #   'response.status_code'
]

# On these headers we will run HashingVectorizer
COMPLEX_HEADERS = [
                    # 'request.headers.User-Agent',
                   #    'request.headers.Set-Cookie',
                   #    'request.headers.Date',
                   #    'request.url',
                #    'response.body',
                #    'response.headers.Location',
                #    'request.headers.Content-Length',
                #    'request.headers.Cookie',
                #    'response.headers.Set-Cookie'
                   ]

COLUMNS_TO_REMOVE = [
    'request.headers.Host',
    'request.headers.Accept',
    'request.headers.Connection',
    'request.headers.Sec-Fetch-User',
    'request.body',
    "request.headers.Sec-Fetch-Mode",
    "request.headers.Accept-Language",
    "request.headers.Set-Cookie",
    "request.headers.Date",
    "response.status_code",
    "response.status",
    'request.url',
    'request.method',
    'request.headers.Content-Length',
    'response.headers.Content-Length',
    'response.headers.Set-Cookie',
    'response.headers.Location',
    'request.headers.User-Agent',
    'request.headers.Cookie',
    'request.headers.Sec-Fetch-Dest',
    'request.headers.Sec-Fetch-Site',
    'request.headers.Accept-Encoding',
    'response.headers.Content-Type',
    'response.body',
]

# This is our main preprocessing function that will iterate over all of the chosen 
# columns and run some feature extraction models
def vectorize_df(df):
    cookie_injection = []
    LOG4J =[]
    log_forging = []
    SQL_Injection = []
    RCE_Injection = []
    Directory_Traversal =[]
    for index, row in df.iterrows():
        if "password=" in row['request.url'] or 'SELECT' in row['request.url']:
            SQL_Injection.append(1)
        else:
            SQL_Injection.append(0)
        if "/cookielogin" in row['request.url']:
            cookie_injection.append(1)
        else:
            cookie_injection.append(0)
        if "%20user%20" in row['request.url']:
            log_forging.append(1)
        else:
            log_forging.append(0)
        if '/etc/passwd.txt' in row['request.url'] or "/windows.ini.txt" in row['request.url']:
            Directory_Traversal.append(1)
        else:
            Directory_Traversal.append(0)
        if "script" in row['request.url'] or "get_flashed_messages" in row['request.url']:
            RCE_Injection.append(1)
        else:
            RCE_Injection.append(0)
        if '{jndi' in row["request.headers.Set-Cookie"] or row['request.headers.Sec-Fetch-Dest'] != "document" or row['request.headers.Sec-Fetch-Site'] != "none" or row['request.headers.Accept-Encoding'] != "gzip, deflate, br":
            LOG4J.append(1)
        else:
            LOG4J.append(0)
        
    df["SQL_Injection"] = SQL_Injection
    df["cookie_injection"] = cookie_injection
    df["log_forging"] = log_forging
    df["RCE_Injection"] = RCE_Injection
    df["Directory_Traversal"] = Directory_Traversal
    df["LOG4J"] = LOG4J
    le = LabelEncoder()
    h_vec = HashingVectorizer(n_features=4)

    # Run LabelEncoder on the chosen features
    for column in SIMPLE_HEADERS:
        df[column] = le.fit_transform(df[column])

    # Run HashingVectorizer on the chosen features
    for column in COMPLEX_HEADERS:
        newHVec = h_vec.fit_transform(df[column])
        df[column] = newHVec.todense()

    # Remove some columns that may be needed.. (Or not, you decide)
    for column in COLUMNS_TO_REMOVE:
        df.drop(column, axis=1, inplace=True)
    return df


df = vectorize_df(df)

# Memory check (For large datasets sometimes the dataframe will exceed the computers resources)
df.info(memory_usage="deep")

# Choose the right features
# In our example code we choose all the columns as our feature this can be the right or wrong way to approach the model, you choose.

features_list = df.columns.to_list()
features_list.remove('label')
features_list.remove('attack_type')
print(features_list)

# Recheck all datatype before training to see we don't have any objects in our features
# In this example our model must get features containing only numbers so we recheck to see if we missed anything during preprocessing
df.dtypes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127500 entries, 0 to 127499
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   attack_type          127500 non-null  object
 1   label                127500 non-null  object
 2   SQL_Injection        127500 non-null  int64 
 3   cookie_injection     127500 non-null  int64 
 4   log_forging          127500 non-null  int64 
 5   RCE_Injection        127500 non-null  int64 
 6   Directory_Traversal  127500 non-null  int64 
 7   LOG4J                127500 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 21.2 MB
['SQL_Injection', 'cookie_injection', 'log_forging', 'RCE_Injection', 'Directory_Traversal', 'LOG4J']


attack_type object
label object
SQL_Injection int64
cookie_injection int64
log_forging int64
RCE_Injection int64
Directory_Traversal int64
LOG4J int64
dtype: object

In [191]:
df

,attack_type,label,SQL_Injection,cookie_injection,log_forging,RCE_Injection,Directory_Traversal,LOG4J
0,Benign,Benign,0,0,0,0,0,0
1,Benign,Benign,0,0,0,0,0,0
2,Benign,Benign,0,0,0,0,0,0
3,Benign,Benign,0,0,0,0,0,0
4,Benign,Benign,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
127495,Benign,Benign,0,0,0,0,0,0
127496,Benign,Benign,0,0,0,0,0,0
127497,Benign,Benign,0,0,0,0,0,0
127498,Benign,Benign,0,0,0,0,0,0


In [192]:
# Data train and test split preparations. Here we will insert our feature list and label list.
# Afterwards the data will be trained and fitted on the amazing XGBoost model
# X_Train and y_Train will be used for training
# X_test and y_test.T will be used for over fitting checking and overall score testing

# We convert the feature list to a numpy array, this is required for the model fitting
X = df[features_list].to_numpy()

# This column is the desired prediction we will train our model on
y = np.stack(df[test_type])

# We split the dataset to train and test according to the required ration
# Do not change the test_size -> you can change anything else
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1765, random_state=42, stratify=y)

# We print the resulted datasets and count the difference 
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
counter = Counter(y)
counter

(104996, 6) (104996,)
(22504, 6) (22504,)


Counter({'Benign': 117914, 'Malware': 9586})

In [193]:
# We choose our model of choice and set it's hyper parameters you can change anything
clf = RandomForestClassifier()

# Train Model
clf.fit(X_train, y_train)

# Check data balance and variety
print(sorted(Counter(y_train).items()))

[('Benign', 97102), ('Malware', 7894)]


In [194]:
# We print our results
sns.set(rc={'figure.figsize': (15, 8)})
predictions = clf.predict(X_test)
true_labels = y_test
list_not_eq =[]
for i in range(len(predictions)):
    if predictions[i] != true_labels[i]:
        print(f'predictions {predictions[i]}, true_labels {true_labels[i]}')
        list_not_eq.append(i)
print(list_not_eq)
print(len(list_not_eq))
cf_matrix = confusion_matrix(true_labels, predictions)
clf_report = classification_report(true_labels, predictions, digits=5)
heatmap = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g',
                      xticklabels=np.unique(true_labels),
                      yticklabels=np.unique(true_labels))

# The heatmap is cool but this is the most important result
print(clf_report)

[]
0
              precision    recall  f1-score   support

      Benign    1.00000   1.00000   1.00000     20812
     Malware    1.00000   1.00000   1.00000      1692

    accuracy                        1.00000     22504
   macro avg    1.00000   1.00000   1.00000     22504
weighted avg    1.00000   1.00000   1.00000     22504



In [121]:
# Now it's your turn, use the model you have just created :)

# Read the valuation json, preprocess it and run your model 
with open(f'combined_datasets_for_students/dataset_{str(dataset_number)}_val.json') as file:
    raw_ds = json.load(file)
test_df = pd.json_normalize(raw_ds, max_level=2)
for column in test_df.columns[test_df.isna().any()].tolist():
    # df.drop(column, axis=1, inplace=True)
    test_df[column] = test_df[column].fillna('None')

# Preprocess the validation dataset, remember that here you don't have the labels
test_df = vectorize_df(test_df)
# Predict with your model
X = test_df[features_list].to_numpy()
predictions = clf.predict(X)
predictions


array(['Benign', 'Benign', 'Benign', ..., 'Benign', 'Benign', 'Benign'],
 dtype='%lt;U7')

In [94]:
test_df

,request.headers.User-Agent,request.headers.Accept-Encoding,request.headers.Accept-Language,request.headers.Sec-Fetch-Site,request.headers.Sec-Fetch-Mode,request.headers.Sec-Fetch-Dest,request.headers.Date,request.method,request.url,response.status,response.headers.Content-Type,response.headers.Content-Length,response.status_code,response.body,request.headers.Cookie,request.headers.Content-Length,response.headers.Location,response.headers.Set-Cookie,SQL_Injection,cookie_injection,log_forging,RCE_Injection,Directory_Traversal,LOG4J
0,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:25...,"gzip, deflate, br",de,none,same-origin,document,"Mon, 21 Nov 2022 18:39:55 GMT",GET,http://127.0.0.1:5000/orders/check/exists/9701,404 NOT FOUND,text/html; charset=utf-8,207,404,<!doctype html>\n<html lang=en>\n<title>404 No...,None,None,None,None,0,0,0,0,0,0
1,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; r...,"gzip, deflate, br",de,none,same-origin,document,"Mon, 21 Nov 2022 18:39:56 GMT",GET,http://127.0.0.1:5000/categories/check/number/...,404 NOT FOUND,application/json,43,404,"{""error"": ""Category number not found""}",None,None,None,None,0,0,0,0,0,0
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; r...,"gzip, deflate, br",de-CH,none,websocket,document,"Mon, 21 Nov 2022 18:39:57 GMT",GET,http://127.0.0.1:5000/states/6885,404 NOT FOUND,application/json,33,404,"{""error"": ""State not found""}",None,None,None,None,0,0,0,0,0,0
3,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,"gzip, deflate, br","en-US,en;q=0.9,he;q=0.8",none,same-origin,document,"Mon, 21 Nov 2022 18:39:57 GMT",GET,http://127.0.0.1:5000/states/10070,404 NOT FOUND,application/json,33,404,"{""error"": ""State not found""}",None,None,None,None,0,0,0,0,0,0
4,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:27...,"gzip, deflate, br",de,none,websocket,document,"Mon, 21 Nov 2022 18:39:58 GMT",GET,http://127.0.0.1:5000/about/greet,200 OK,text/html; charset=utf-8,664,200,<p>If you get stuck or just want to check your...,None,None,None,None,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,Mozilla/5.0 (Windows NT 6.1; rv:27.3) Gecko/20...,"gzip, deflate, br","en-US,en;q=0.5",none,same-origin,document,"Mon, 21 Nov 2022 18:49:38 GMT",GET,http://127.0.0.1:5000/categories/check/all,200 OK,application/json,1112,200,"[{""category_id"": ""1"", ""category_name"": ""Bevera...",None,None,None,None,0,0,0,0,0,0
751,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,"gzip, deflate, br","en-US,en;q=0.9,he;q=0.8",none,same-origin,document,"Mon, 21 Nov 2022 18:49:38 GMT",GET,http://127.0.0.1:5000/cookielogin,200 OK,text/html; charset=utf-8,104,200,<h1>Logged in as Jewel</h1><form method='POST'...,username=gASVyAAAAAAAAACMCGJ1aWx0aW5zlIwEZXZhb...,None,None,None,0,1,0,0,0,0
752,Mozilla/5.0 (X11; Linux i586; rv:31.0) Gecko/2...,"gzip, deflate, br",de,none,websocket,document,"Mon, 21 Nov 2022 18:49:40 GMT",GET,http://127.0.0.1:5000/about/random_page,200 OK,text/html; charset=utf-8,1163,200,"<figure><img src=""https://img-c.udemycdn.com/r...",None,None,None,None,0,0,0,0,0,0
753,Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKi...,"gzip, deflate, br",de,none,same-origin,document,"Mon, 21 Nov 2022 18:49:40 GMT",GET,http://127.0.0.1:5000/cookielogin,200 OK,text/html; charset=utf-8,105,200,<h1>Logged in as Martin</h1><form method='POST...,username=gASVyQAAAAAAAACMCGJ1aWx0aW5zlIwEZXZhb...,None,None,None,0,1,0,0,0,0


In [122]:
# Save your preditions
enc = LabelEncoder()
np.savetxt(f'results/dataset_{str(dataset_number)}_{test_type}_result.txt', enc.fit_transform(predictions), fmt='%2d')